¿Qué es TextRank?

A diferencia de simplemente tomar el promedio de los valores TF-IDF. TextRank se inspira en cómo las oraciones están relacionadas entre sí en términos de contenido.

La idea central de TextRank proviene del algoritmo PageRank de Google. PageRank funciona de la siguiente manera:
* Cada página web se considera un-nodo en una red.
* Se asigna una puntuación a cada página web basada en la cantidad y calidad de los enlaces entrantes.
* El internet se visualiza como una red de páginas web interconectadas.

La idea detrás de PageRank es que una página es importante si otras páginas importantes la enlazan. Para adaptar el concepto de PageRank al texto:  
**1. Preprocesar el documento**: Tokenizar el documento en oraciones. Se tratan las oraciones como "páginas web".  
**2. Vectorizar las oraciones o palabras** con el fin de hacer una representación basada en grafos. Esto podría hacerse usando TF-IDF, bolsa de palabras, etc.  
**3. Crear una matriz de similitud**: donde cada elemento representen oraciones o palabras. Los "enlaces" entre oraciones se determinan mediante la similitud del contenido (cosine similarity de sus vectores TF-IDF).  
**4. Normalizar**: asegurarse de que cada fila sume uno.  
**5. Construir un grafo**: Usando la matriz de similitud, crear un grafo donde los nodos representen oraciones o palabras y los bordes representen puntos de similitud.  
**6. Puntuar cada nodo**: mediante el mecanismo de clasificación TextRank. Las oraciones que son similares a muchas otras oraciones "importantes" obtienen una puntuación más alta.  
**7. Suavizado y regularización**, que cada nodo tenga aunque sea una pequeñisima probabilidad de apuntar a cualquier otra frase, para evitar sesgo.  
**8. Extraer resultados**:Para resumir, seleccionar las oraciones con la mejor clasificación. Para la extracción de palabras clave, elegir las palabras mejor clasificadas.

In [2]:
import pandas as pd
import numpy as np

# Prevenir que no se corten las oraciones entre palabras debido a saltos de línea, sino en palabras completas
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# nltk.download('punkt')
# nltk.download('stopwords')

In [4]:
df = pd.read_csv('df_total.csv')
df.head()

,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra


In [62]:
doc = df['news'].sample()
print(doc.iloc[0])

La semana pasada se conoció que el presidente Gustavo Petro intervendría a la Comisión de Regulación de Energía y Gas Creg después de las sorpresivas alzas que tuvieron algunos servicios públicos según el último informe del IPC revelado por el Dane tuvieron un aumento de 259 anual hasta agosto.Las comisiones reguladoras son para regular los mercados en función del derecho universal no para afirmar procesos de especulación financiera como hasta ahora lo han realizado comentó al respecto el mandatario. Sobre lo que se espera que sean las medidas que tome el Presidente frente a la Creg aún no hay una decisión fija. Pero analistas dicen que una de las acciones puede dirigirse por la forma en la que se establecen los precios.Si bien la tarifa es regulada por la comisión y esta define qué y cómo es lo que se puede cobrar por cada uno de los rubros para la prestación del servicio esta no dice a qué se ata dicho aumento como sí se hace en otros casos. Esto es lo que vemos en contratos de cualq

In [64]:
def wrap(x):
    return textwrap.fill(doc.iloc[0], replace_whitespace=False, fix_sentence_endings=True)

In [65]:
doc2 = wrap(doc.iloc[0])

In [66]:
lineas = doc2.split(".")

In [67]:
print(lineas)

['La semana pasada se conoció que el presidente Gustavo Petro\nintervendría a la Comisión de Regulación de Energía y Gas Creg después\nde las sorpresivas alzas que tuvieron algunos servicios públicos según\nel último informe del IPC revelado por el Dane tuvieron un aumento de\n259 anual hasta agosto', 'Las comisiones reguladoras son para regular los\nmercados en función del derecho universal no para afirmar procesos de\nespeculación financiera como hasta ahora lo han realizado comentó al\nrespecto el mandatario', '  Sobre lo que se espera que sean las medidas\nque tome el Presidente frente a la Creg aún no hay una decisión fija', '\nPero analistas dicen que una de las acciones puede dirigirse por la\nforma en la que se establecen los precios', 'Si bien la tarifa es\nregulada por la comisión y esta define qué y cómo es lo que se puede\ncobrar por cada uno de los rubros para la prestación del servicio esta\nno dice a qué se ata dicho aumento como sí se hace en otros casos', '\nEsto es lo

In [68]:
# Eliminar elementos vacíos para prevenir errores
lineas = [item for item in lineas if item.strip()]

In [69]:
print(lineas)

['La semana pasada se conoció que el presidente Gustavo Petro\nintervendría a la Comisión de Regulación de Energía y Gas Creg después\nde las sorpresivas alzas que tuvieron algunos servicios públicos según\nel último informe del IPC revelado por el Dane tuvieron un aumento de\n259 anual hasta agosto', 'Las comisiones reguladoras son para regular los\nmercados en función del derecho universal no para afirmar procesos de\nespeculación financiera como hasta ahora lo han realizado comentó al\nrespecto el mandatario', '  Sobre lo que se espera que sean las medidas\nque tome el Presidente frente a la Creg aún no hay una decisión fija', '\nPero analistas dicen que una de las acciones puede dirigirse por la\nforma en la que se establecen los precios', 'Si bien la tarifa es\nregulada por la comisión y esta define qué y cómo es lo que se puede\ncobrar por cada uno de los rubros para la prestación del servicio esta\nno dice a qué se ata dicho aumento como sí se hace en otros casos', '\nEsto es lo

In [70]:
len(lineas)

36

In [71]:
tokenizar = TfidfVectorizer(
    stop_words=stopwords.words('spanish'),
    norm='l1',
)

In [72]:
X = tokenizar.fit_transform(lineas)
X

<36x402 sparse matrix of type '<class 'numpy.float64'>'
	with 600 stored elements in Compressed Sparse Row format>

In [73]:
# Matriz de similitud
S = cosine_similarity(X)

In [74]:
S.shape

(36, 36)

In [75]:
S

array([[1.        , 0.        , 0.06153979, ..., 0.07421124, 0.11334038,
        0.06175058],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06153979, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07421124, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.11334038, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.06175058, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [76]:
# Normalize similirity matrix, sumar elemento de la fila por la suma de toda esa fila
S = S/S.sum(axis=1, keepdims= True)
S

array([[0.39211975, 0.        , 0.02413097, ..., 0.02909969, 0.044443  ,
        0.02421362],
       [0.        , 0.75584118, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03401323, 0.        , 0.55270302, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.04755671, 0.        , 0.        , ..., 0.64082898, 0.        ,
        0.        ],
       [0.06842608, 0.        , 0.        , ..., 0.        , 0.60372201,
        0.        ],
       [0.0308693 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.49990307]])

In [77]:
S[1].sum()

1.0

In [78]:
# Puntar nodos como un proceso de Markov, para ello convertir en una matriz de transición
# es decir, añadir una pequeña probabilidad de que cualquier frase pueda pasar a otra en el lugar de los 0

# Matriz dummy de 1's de las mismas dimensiones que la de similaridad
U = np.ones_like(S)
U

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [79]:
U.shape

(36, 36)

In [80]:
# Matriz de transición uniforme, donde cada línea sume 1
U = np.ones_like(S) / len(S)
U

array([[0.02777778, 0.02777778, 0.02777778, ..., 0.02777778, 0.02777778,
        0.02777778],
       [0.02777778, 0.02777778, 0.02777778, ..., 0.02777778, 0.02777778,
        0.02777778],
       [0.02777778, 0.02777778, 0.02777778, ..., 0.02777778, 0.02777778,
        0.02777778],
       ...,
       [0.02777778, 0.02777778, 0.02777778, ..., 0.02777778, 0.02777778,
        0.02777778],
       [0.02777778, 0.02777778, 0.02777778, ..., 0.02777778, 0.02777778,
        0.02777778],
       [0.02777778, 0.02777778, 0.02777778, ..., 0.02777778, 0.02777778,
        0.02777778]])

In [81]:
# Conbinar esta segunda matriz dummy con la matriz de similitud para obtenerla suavizada (sin 0's)
factor = 0.15 # número pequeño ya que será el porcentaje que la matriz dummy modifique a la matriz de similitud
S = (1 - factor)*S + factor * U

In [82]:
S

array([[0.33746846, 0.00416667, 0.02467799, ..., 0.0289014 , 0.04194322,
        0.02474824],
       [0.00416667, 0.64663167, 0.00416667, ..., 0.00416667, 0.00416667,
        0.00416667],
       [0.03307791, 0.00416667, 0.47396424, ..., 0.00416667, 0.00416667,
        0.00416667],
       ...,
       [0.04458987, 0.00416667, 0.00416667, ..., 0.5488713 , 0.00416667,
        0.00416667],
       [0.06232884, 0.00416667, 0.00416667, ..., 0.00416667, 0.51733038,
        0.00416667],
       [0.03040557, 0.00416667, 0.00416667, ..., 0.00416667, 0.00416667,
        0.42908427]])

In [83]:
eigenvals, eigenvecs = np.linalg.eig(S.T)

In [84]:
# El valor impportante es el primer 1
eigenvals

array([1.        , 0.85      , 0.77400565, 0.70346684, 0.22377164,
       0.68776778, 0.6772341 , 0.64796637, 0.25558682, 0.63324888,
       0.61563713, 0.27283839, 0.28810645, 0.29653337, 0.30424792,
       0.58211062, 0.57460644, 0.55233612, 0.53761511, 0.53070168,
       0.32670312, 0.49806819, 0.34072537, 0.3485133 , 0.35994276,
       0.36678935, 0.37620594, 0.38764601, 0.40065786, 0.41073244,
       0.47828096, 0.47139894, 0.43290502, 0.45241596, 0.44925714,
       0.44577354])

In [85]:
# Son las relaciones del 1 con el resto de frases
eigenvecs[:,0]

array([0.210356  , 0.13549242, 0.15823548, 0.14098944, 0.17546431,
       0.16552526, 0.17187817, 0.16673559, 0.18189314, 0.16246075,
       0.17888612, 0.14140944, 0.1377931 , 0.14343739, 0.1745774 ,
       0.17018666, 0.15767726, 0.18228709, 0.15965784, 0.21818146,
       0.1347882 , 0.16390286, 0.16597653, 0.17302736, 0.19642243,
       0.17172316, 0.14804181, 0.15483598, 0.14995677, 0.15901447,
       0.16188413, 0.19491153, 0.18057878, 0.14679318, 0.15163311,
       0.17229488])

In [86]:
scores = eigenvecs[:,0]

In [87]:
sort_idx = np.argsort(-scores)

In [88]:
# Ordenar las oraciones más importantes, ranqueo de eigenvectores de mayor a menor
sort_idx

array([19,  0, 24, 31, 17,  8, 32, 10,  4, 14, 23, 35,  6, 25, 15,  7, 22,
        5, 21,  9, 30, 18, 29,  2, 16, 27, 34, 28, 26, 33, 13, 11,  3, 12,
        1, 20], dtype=int64)

In [89]:
print("Crear un resumen:")
for i in sort_idx[:5]:
    print(wrap("%.2f: %s" % (scores[i], lineas[i])))

Crear un resumen:
La semana pasada se conoció que el presidente Gustavo Petro
intervendría a la Comisión de Regulación de Energía y Gas Creg después
de las sorpresivas alzas que tuvieron algunos servicios públicos según
el último informe del IPC revelado por el Dane tuvieron un aumento de
259 anual hasta agosto.Las comisiones reguladoras son para regular los
mercados en función del derecho universal no para afirmar procesos de
especulación financiera como hasta ahora lo han realizado comentó al
respecto el mandatario.  Sobre lo que se espera que sean las medidas
que tome el Presidente frente a la Creg aún no hay una decisión fija.
Pero analistas dicen que una de las acciones puede dirigirse por la
forma en la que se establecen los precios.Si bien la tarifa es
regulada por la comisión y esta define qué y cómo es lo que se puede
cobrar por cada uno de los rubros para la prestación del servicio esta
no dice a qué se ata dicho aumento como sí se hace en otros casos.
Esto es lo que vemos en